# Extracción de datos Datasets Google Maps y YELP

### Importaciones

In [1]:
import pandas as pd
import numpy as np
import json
import datetime
import os

### Utiles

In [5]:

state_abreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", 
    "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", 
    "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", 
    "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", 
    "WI", "WY"
]

state_dictionary = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", 
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", 
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", 
    "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", 
    "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts", "MI": "Michigan", 
    "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri", "MT": "Montana", 
    "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey", 
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", 
    "OH": "Ohio", "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", 
    "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota", "TN": "Tennessee", 
    "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington", 
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}

In [6]:
import os

generated_dir = "Generated"

os.mkdir(generated_dir)
os.chdir(generated_dir)
os.mkdir('Google')
os.mkdir('Yelp')
os.chdir('../')

os.getcwd()

'z:\\PF_DATAPT07'

# 1. Extracción  con los datasets de Google Maps

## 1.1 Metada de Sitios

Recorremos todo el directorio, lo hacemos archivo por archivo y línea por línea ya que no se puede abrir directamente los archivos por su dimensión y porque no están en formato de array, sino están constituidos en un registro por lìnea.
Durante la lectura filtramos los que incluyan <code>Restaurant</code> en el campo de categoría, para alivianar el dataset final.

In [7]:
# Tiempo de Demora Medio: 51 segundos.
lineas_json = []

# Son 11 archivos con un ordinal, del 1 al 11
for i in range(1, 12):
    path = f'Datasets/Google Maps/metadata-sitios/{i}.json'
    with open(path, 'r') as file:
        for l in file:
            try:
                linea_j = json.loads(l)
                if 'restaurant' in " ".join(linea_j['category']).lower():
                    lineas_json.append(linea_j)
            except:
                pass

df = pd.DataFrame(lineas_json)

df.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...","{'Service options': ['Outdoor seating', 'Curbs...",Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
2,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,"[Bakery, Health food restaurant]",4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...",{'Service options': ['Delivery']},Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...


In [8]:
df.loc[5, 'address']

'Cape Seafood Shack, 603 Del Prado Blvd S, Cape Coral, FL 33990'

Exportamos a formato Parquet

In [11]:
df.to_parquet(r'Generated\Google\metada_sitios.parquet')

Tamaño Directorio <code>metadata-sitios</code>: 2.76 Gb

Tamaño Archivo <code>metada_sitios.parquet</code>: 60.43 Mb


In [ ]:
df.info()

Dimensiones: 212.014 filas x 15 Columnas

## 1.1.1 Obtención de información de Estados

En base al campo <code>Address</code> obtenemos el estado donde se encuentra el negocio. Nos servirá para luego seleccionar los estados con más restaurantes.

In [12]:
def get_state_ab(st):
    try:
        state = st.split(', ')[-1].split(' ')[0]
        if state in state_abreviations:
            return state
        else:
            return np.nan
    except:
        return np.nan
    
df['state_ab'] = df['address'].apply(get_state_ab)

De esta manera conseguimos el top 5 de los estados con más restaurantes

In [13]:
top_5 = df['state_ab'].value_counts().head(5).index.to_list()

In [14]:
top_5

['CA', 'TX', 'NY', 'FL', 'PA']

Completamos el campo estado que es más descriptivo

In [15]:
df['us_state'] = df['state_ab'].map(state_dictionary)

In [16]:
df['us_state'].head(5)

0      California
1      California
2            Iowa
3    Pennsylvania
4          Hawaii
Name: us_state, dtype: object

Obtenemos un arreglo de URLs de los archivos correspondientes para cada estado del top 5, con el fin de extraer los datos en un bucle.

In [17]:
top_5_url = [f"Datasets/Google Maps/reviews-estados/review-{state_dictionary[i].replace(' ', '_')}/" for i in top_5]

top_5_url


['Datasets/Google Maps/reviews-estados/review-California/',
 'Datasets/Google Maps/reviews-estados/review-Texas/',
 'Datasets/Google Maps/reviews-estados/review-New_York/',
 'Datasets/Google Maps/reviews-estados/review-Florida/',
 'Datasets/Google Maps/reviews-estados/review-Pennsylvania/']

Creamos un diccionario con la cantidad de archivos por cada directorio de estados, con el fin de utilizarlo en un bucle en la extracción de datos.

In [18]:
cantidad_archivos = {}

for i in top_5_url:
    for j in os.walk(i):
        cantidad_archivos[i] = len(j[2])

cantidad_archivos

{'Datasets/Google Maps/reviews-estados/review-California/': 18,
 'Datasets/Google Maps/reviews-estados/review-Texas/': 14,
 'Datasets/Google Maps/reviews-estados/review-New_York/': 18,
 'Datasets/Google Maps/reviews-estados/review-Florida/': 19,
 'Datasets/Google Maps/reviews-estados/review-Pennsylvania/': 15}

Comprobación

In [19]:
for i in os.walk('Datasets/Google Maps/reviews-estados/review-Pennsylvania'):
    print(len(i[2]))

15


In [20]:
cantidad_archivos['Datasets/Google Maps/reviews-estados/review-Pennsylvania/']

15

## 1.2 Reviews Estados

Ya con los estados elegidos estamos en condiciones de ingestar los datos de las carpetas correspondientes dentro del directorio <code>reviews-estados</code>.
Es información masiva lo que genera un archivo de grandes dimensiones, sin embargo previamente filtramos por el parámetro de año <code>2017-2019</code> valiéndonos del campo <code>time</code>, que tiene es un <code>timestamp</code>, pero con 3 digitos más que el usado por <code>datetime</code> de Python. Le agregamos el campo <code>Estado</code> que es más descriptivo.

In [22]:
### Demora 7 minutos y 40 segundos, 11 minutos, varía

lineas_json_revs_google = []

for i in top_5_url:
    count = 0
    for c in range(1,cantidad_archivos[i]+1):
        with open(str(i)+str(c)+".json", 'r', encoding='utf-8') as f:        
            for s in f:
                linea = json.loads(s)
                linea['anio'] = datetime.datetime.fromtimestamp(linea['time']/1000).year
                linea['estado'] = i.split('-')[-1][:-1]
                
                if linea['anio'] in [2017,2018,2019]:
                    lineas_json_revs_google.append(linea)

df_revs_google = pd.DataFrame(lineas_json_revs_google)

df_revs_google.head(3)

In [ ]:
merge_site_reviews = pd.merge(df_revs_google, df, left_on='gmap_id', right_on='gmap_id')

In [ ]:
merge_site_reviews

In [ ]:
merge_site_reviews.to_parquet(r'Generated\Google\merge_site_reviews.parquet')

In [ ]:
df_revs_google.to_parquet(r'Generated\Google\reviews-estados.parquet')

Tamaño archivo: 760 Mb

Tamaño dataset: 24.3 Gb

In [ ]:
df_revs_google.info()

Tamaño 8.339.179 filas x 10 Columnas.

# 2. Extracción de los Dataset de YELP

### 2.1 Business

Contiene los datos de las entidades negocios de Yelp, a un primer vistazo tiene las columnas duplicadas, por lo que hay que hacer un recorte, ya que la segunda mitad tiene datos vacíos en su inmensa mayoría.

In [ ]:
url_business = r'Datasets\Yelp\business.pkl'

df_business = pd.read_pickle(url_business)

df_business = df_business.iloc[:,:-14]

In [ ]:
df_business.sample(3)

In [ ]:
df_business.shape

Luego con la ayuda del campo <code>state</code> filtramos los negocios que se encuentran en los estados seleccionados en nuestro análisis.

In [ ]:
df_business = df_business[df_business.state.isin(top_5)]

In [ ]:
df_business.info()

Seguimos filtrando a través del campo <code>categories</code>, para obtener los negocios que son restaurantes.

In [ ]:
def is_restaurant(st):
    try: 
        test = "".join(st).lower()
        return 'restaurant' in test
    except:
        return False

df_business = df_business[df_business['categories'].apply(is_restaurant)]

In [ ]:
df_business.info()

In [ ]:
df_business.to_parquet(r'Generated\Yelp\bussines.parquet')

### 2.2 Checkin

In [ ]:
lineas_json = []
path_checkin = r'Datasets\Yelp\checkin.json'
with open(path_checkin, 'r', encoding='utf-8') as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j['date'][:4]
            # if 'restaurant' in " ".join(linea_j['category']).lower():
            if anio in ['2017', '2018', '2019']:
                lineas_json.append(linea_j)
        except:
            pass

df_checkin = pd.DataFrame(lineas_json)

In [ ]:
df_checkin

In [ ]:
merge_business_checkin = pd.merge(df_business, df_checkin, left_on='business_id', right_on='business_id')

In [ ]:
merge_business_checkin.info()

In [ ]:
df_checkin.to_parquet(r'Generated\Yelp\checkin.parquet')
merge_business_checkin.to_parquet(r'Generated\YELP\business_checkin.parquet')

### 2.3 Tips 

Realizamos la extracción de los datos y filtramos por año según nuestro análisis.

In [ ]:
lineas_json = []
path_tip = r'Datasets\Yelp\tip.json'
with open(path_tip, 'r', encoding='utf-8') as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j['date'][:4]
            if anio in ['2017', '2018', '2019']:
                lineas_json.append(linea_j)
        except:
            pass

df_tip = pd.DataFrame(lineas_json)


In [ ]:
df_tip.sample(5)

In [ ]:
df_tip.to_parquet(r'Generated\Yelp\tip.parquet')

Unimos el el dataframe de tips con el de negocios

In [ ]:
tips_merged = pd.merge(df_tip, df_business, left_on='business_id', right_on='business_id')

In [ ]:
tips_merged.sort_values('business_id').head(3)

In [ ]:
tips_merged.to_parquet(r'Generated\Yelp\business_tip.parquet')

### 2.4 Review

In [ ]:
df_reviews_url =  r'Datasets\Yelp\review.json'

Usamos el mismo método de linea por linea, y en el proceso filtramos por año y por las reseñas que han sido votadas como útiles.

In [ ]:
# 44 segundos

lineas_json_review = []

with open(df_reviews_url, 'r', encoding='utf-8') as f:
    count = 0
    for i in f:
        linea = json.loads(i)
        anio = linea['date'][:4]
        if anio in ['2017', '2018', '2019'] and linea['useful'] == 1:    
            lineas_json_review.append(linea)


df_reviews = pd.DataFrame(lineas_json_review)

In [ ]:
df_reviews.sample(10)

In [ ]:
df_reviews.info()

Aligeramos el dataset con unos downgrades de tipo de variables.

In [ ]:
df_reviews['funny'] = df_reviews['funny'].astype('int8')
df_reviews['stars'] = df_reviews['stars'].astype('int8')
df_reviews['cool'] = df_reviews['cool'].astype('int8')

df_reviews.drop('useful', axis=1, inplace=True, errors='ignore')


In [ ]:
df_reviews.to_parquet(r'Generated\Yelp\review.parquet')

### 2.4 Users Yelp

In [ ]:
import pyarrow.parquet as pq


parquet_file = pq.ParquetFile(r'Datasets\Yelp\user.parquet')

arr_df = []

for batch in parquet_file.iter_batches():
    # count = count +1
    batch_df = batch.to_pandas()
    batch_df['elite'] = batch_df['elite'].apply(lambda x: x.split(','))
    batch_df['elite_len'] = batch_df['elite'].apply(lambda x: len(x))
    batch_df = batch_df.query("elite_len > 1")
    arr_df.append(batch_df)

df_users = pd.concat(arr_df)

In [ ]:
df_users.sample(5)

In [ ]:
df_users.info()

In [ ]:
df_users.reset_index(inplace=True)
df_users.drop('index', axis=1, inplace=True, errors="ignore")

In [ ]:
df_users.to_parquet(r"Generated\Yelp\users_extracted.parquet")

## NORMALIZACIÓN DE DATOS

In [2]:
import pandas as pd
from datetime import datetime

### **GOOGLE**

#### *1. MERGE_SITE_REVIEWS*

In [89]:
df_siterev = pd.read_parquet("Generated\Google\merge_site_reviews.parquet")

In [94]:
df_siterev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393452 entries, 0 to 2393451
Data columns (total 26 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   name_x            object 
 2   time              int64  
 3   rating            int64  
 4   text              object 
 5   pics              object 
 6   resp              object 
 7   gmap_id           object 
 8   anio              int64  
 9   estado            object 
 10  name_y            object 
 11  address           object 
 12  description       object 
 13  latitude          float64
 14  longitude         float64
 15  category          object 
 16  avg_rating        float64
 17  num_of_reviews    int64  
 18  price             object 
 19  hours             object 
 20  MISC              object 
 21  state             object 
 22  relative_results  object 
 23  url               object 
 24  state_ab          object 
 25  us_state          object 
dtypes: float64(3),

In [19]:
df_siterev.sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,state_ab,us_state
348493,107942381515148867431,Sandy P,1529050068250,5,None,None,None,0x8085612890558ddf:0x125ed0ca2f98a892,2018,California,...,3.6,1078,₩,"[[Wednesday, 5AM–2AM], [Thursday, 5AM–12AM], [...",{'Accessibility': ['Wheelchair-accessible car ...,Open ⋅ Closes 2AM,"[0x808561288074c9d9:0x129e8b6ca0ddc163, 0x8085...",https://www.google.com/maps/place//data=!4m2!3...,None,None
1972544,108973483426689098114,j b,1552163444809,4,None,None,None,0x88e77010a00e128d:0x761d451cbcaed653,2019,Florida,...,4.5,1772,₩₩,"[[Saturday, 7AM–5PM], [Sunday, 8AM–5PM], [Mond...",{'Accessibility': ['Wheelchair-accessible car ...,Open ⋅ Closes 5PM,"[0x88e7701a0633b12d:0xd328ea95e4f2b0fc, 0x88e7...",https://www.google.com/maps/place//data=!4m2!3...,None,None
365299,117154565760944188399,MICHAEL Bansuelo,1517295312556,5,None,None,None,0x80c2c0aeee51ffb5:0xb2f24102f8a608c8,2018,California,...,4.4,688,₩₩,"[[Wednesday, 9:30AM–10:30PM], [Thursday, 9:30A...",{'Accessibility': ['Wheelchair-accessible entr...,Closed ⋅ Opens 9:30AM,"[0x80c2c12c56569857:0xb2307fc2d0d613c, 0x80c2b...",https://www.google.com/maps/place//data=!4m2!3...,None,None


In [20]:
df_siterev.columns.values

array(['user_id', 'name_x', 'time', 'rating', 'text', 'pics', 'resp',
       'gmap_id', 'anio', 'estado', 'name_y', 'address', 'description',
       'latitude', 'longitude', 'category', 'avg_rating',
       'num_of_reviews', 'price', 'hours', 'MISC', 'state',
       'relative_results', 'url', 'state_ab', 'us_state'], dtype=object)

#### ***~ VERIFICACIÓN DE DUPLICADOS***

In [ ]:
df_siterev['duplex'] = df_siterev.duplicated(subset=['user_id','gmap_id','anio','mes','dia','hora'])
df_siterev[df_siterev.duplex].sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,state,relative_results,url,state_ab,us_state,dtfmt,mes,dia,hora,duplex
2102952,110929799662088863738,Carmine Cannoli,1573992097196,5,excellent cold pressed concoctions as well as ...,None,"{'text': 'Hi Arthur, Thank you so much for vi...",0x89c69123824041c7:0x3da1ff043efcf8e7,2019,Pennsylvania,...,Open ⋅ Closes 8PM,"[0x89c6909c21ba1595:0x4a823e780464b365, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...,PA,Pennsylvania,2019-11-17 12:01:37.196,11,17,12,True
1145604,107690690451688607534,John Perez,1559400302012,5,"great food, ambience, and drinks",None,None,0x89c28b4ecbc8d561:0xef772c6c75e0f520,2019,New_York,...,Open ⋅ Closes 10PM,"[0x89c2f4c85ae4ccc3:0x8380dd7e5b2490fc, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2019-06-01 14:45:02.012,6,1,14,True
1938324,112863588075588246459,Macy,1569154022471,5,they have amazing pizza and much more. they ar...,None,None,0x88db42498f958857:0x7843ef317d6f922,2019,Florida,...,Closed ⋅ Opens 11AM,"[0x88db43d6a79c5613:0xda575757a2490bed, 0x88db...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2019-09-22 12:07:02.471,9,22,12,True


In [ ]:
df_siterev.drop_duplicates(subset=['user_id','gmap_id','anio','mes','dia','hora'],inplace=True)

In [8]:
df_siterev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393452 entries, 0 to 2393451
Data columns (total 26 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   name_x            object 
 2   time              int64  
 3   rating            int64  
 4   text              object 
 5   pics              object 
 6   resp              object 
 7   gmap_id           object 
 8   anio              int64  
 9   estado            object 
 10  name_y            object 
 11  address           object 
 12  description       object 
 13  latitude          float64
 14  longitude         float64
 15  category          object 
 16  avg_rating        float64
 17  num_of_reviews    int64  
 18  price             object 
 19  hours             object 
 20  MISC              object 
 21  state             object 
 22  relative_results  object 
 23  url               object 
 24  state_ab          object 
 25  us_state          object 
dtypes: float64(3),

In [7]:
df_siterev[~df_siterev.pics.isnull()].sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,state_ab,us_state
366219,115191459158252243116,Sarah Bal,1572469393340,4,I have been wanting to review this place for a...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x808e34cc915bd159:0x659ec9ee6650efc6,2019,California,...,4.4,1004,₩₩,"[[Wednesday, 11AM–8PM], [Thursday, 11AM–8PM], ...",{'Accessibility': ['Wheelchair-accessible car ...,Closed ⋅ Opens 11AM,"[0x808e352a573a1a49:0xe805950681af6c8d, 0x808e...",https://www.google.com/maps/place//data=!4m2!3...,None,None
1463632,118251272936885295934,andricci romero,1566164123252,1,"(Translated by Google) Super dissatisfied, it ...",[{'url': ['https://lh5.googleusercontent.com/p...,None,0x88e77d85a70512f1:0xf1404b79022cfbca,2019,Florida,...,2.7,48,None,"[[Monday, Closed], [Tuesday, 5PM–12AM], [Wedne...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 5PM Tue,"[0x88e77c4c9e301503:0x9c555cd78c12d447, 0x88e7...",https://www.google.com/maps/place//data=!4m2!3...,FL,Florida
1532735,114771237847844057444,Ron Kish,1570223395559,4,Tried the pizza 1st trip.Good stuff.Will be back,[{'url': ['https://lh5.googleusercontent.com/p...,{'text': 'We're looking forward to that 2nd vi...,0x88c29402cebd3b25:0xe05511b81b2b8fe9,2019,Florida,...,4.2,468,$$,"[[Saturday, 11AM–8PM], [Sunday, 11AM–8PM], [Mo...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM,"[0x88c293f7b0e06b11:0x155d18d0ebde0896, 0x88c2...",https://www.google.com/maps/place//data=!4m2!3...,FL,Florida


In [27]:
num_nulls_pics = df_siterev.pics.isnull().sum()
num_nulls_pics

2307136

In [28]:
num_nulls_resp = df_siterev.resp.isnull().sum()
num_nulls_resp

2178044

In [3]:
df_siterev['dtfmt'] = df_siterev.time.apply(lambda x: datetime.utcfromtimestamp(x / 1000))
df_siterev['mes']  = df_siterev.dtfmt.dt.month
df_siterev['dia']  = df_siterev.dtfmt.dt.day
df_siterev['hora'] = df_siterev.dtfmt.dt.hour
df_siterev.drop(columns=['dtfmt','duplex'])
df_siterev.name_x = df_siterev.name_x.str.title()
df_siterev.text = df_siterev.text.str.lower()
df_siterev.text = df_siterev.category.str.lower()

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,MISC,state,relative_results,url,state_ab,us_state,dtfmt,mes,dia,hora
2297455,110622153058905199268,Jim Shea,1548426341699,5,i have yet to try something that i didnt like....,None,{'text': 'Thanks for the positive feedback Jim...,0x89c4dbe33c619dc9:0xbacdd627f4a0ea4f,2019,Pennsylvania,...,"{'Accessibility': None, 'Amenities': ['Good fo...",Closes soon ⋅ 9PM ⋅ Opens 11AM Sat,"[0x89c4d95812b9fbb7:0x18484a2597e7ef89, 0x89c4...",https://www.google.com/maps/place//data=!4m2!3...,PA,Pennsylvania,2019-01-25 14:25:41.699,1,25,14
1131038,114897101857765595557,Jeannette R,1569009249173,5,None,None,None,0x89c2590aa9dc256d:0xdcce3a99f1dec72c,2019,New_York,...,"{'Accessibility': None, 'Amenities': ['Good fo...",Permanently closed,[0x89c25f3f5b5f4701:0xbe36f83c2fc9ece2],https://www.google.com/maps/place//data=!4m2!3...,NY,New York,2019-09-20 19:54:09.173,9,20,19
843528,103935041101974043495,Rudhran Mehra,1577761857699,1,None,None,None,0x864c174399ba2671:0x93673c6d02a0bd83,2019,Texas,...,{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11:30AM,"[0x864c1309f0de960d:0x95468394b7d16951, 0x864c...",https://www.google.com/maps/place//data=!4m2!3...,TX,Texas,2019-12-31 03:10:57.699,12,31,3


In [17]:
## determinar si se va a grabar los nuevos parquets ya procesados o en qué momento se graban?
#df_siterev.to_parquet("Generated\Google\merge_site_reviews_nsd.parquet",index=False)

#### ***<u>COMENTARIO</u>***
Las columnas **pics** y **resp** deberían eliminarse porque la cantidades de datos nulos sobrepasa los 90%.

*Se obtuvo las columnas de año, mes, día y hora para posteriores análisis*

#### *2. METADATA_SITIOS*

In [62]:
df_metasit = pd.read_parquet("Generated\Google\metada_sitios.parquet")

#### ***~ VERIFICACIÓN DE DUPLICADOS***

In [63]:
df_metasit['duplex'] = df_metasit.duplicated(subset=['gmap_id'])
df_metasit[df_metasit.duplex]

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,duplex
1061,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,True
1062,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...,True
1063,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,"[Bakery, Health food restaurant]",4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...","{'Accessibility': None, 'Amenities': None, 'At...",Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...,True
1064,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,None,None,"{'Accessibility': None, 'Amenities': ['Good fo...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...,True
1065,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,21.637796,-157.920714,[Restaurant],4.4,18,None,"[[Thursday, 11AM–8PM], [Friday, 11AM–8PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM,"[0x7c00451360f80cf1:0x930291a38bab3132, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,"Spicy Cajun Cafe (BEST Chicken, Burgers, Beign...","Spicy Cajun Cafe (BEST Chicken, Burgers, Beign...",0x86236042637df67f:0x656dd449cec3aedf,None,30.109789,-92.124469,"[Convenience store, Convenience stores organiz...",4.7,18,None,"[[Tuesday, 5AM–9PM], [Wednesday, 5AM–9PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 5AM Wed,"[0x86236042bd6449c7:0xa66964d7cee492e9, 0x8623...",https://www.google.com/maps/place//data=!4m2!3...,True
2118,Carolina Rib King,"Carolina Rib King, 2730 Cannons Campground Rd,...",0x885774cee0afffff:0xf093c7750948ab35,None,35.001732,-81.874038,[Barbecue restaurant],5.0,6,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x88577667a96ecf81:0xb373c8243235954f, 0x8857...",https://www.google.com/maps/place//data=!4m2!3...,True
2119,Taco Truck Bar and Grill,"Taco Truck Bar and Grill, 2737 Cherokee St, St...",0x87d8b473f54a5c5b:0x435f2a4495096c0d,None,38.593842,-90.229106,[Mexican restaurant],3.6,18,$,"[[Tuesday, 3–10PM], [Wednesday, 3–10PM], [Thur...","{'Accessibility': None, 'Amenities': ['Good fo...",Permanently closed,"[0x87d8b38a7656662f:0x679d3a93c96ab599, 0x87d8...",https://www.google.com/maps/place//data=!4m2!3...,True
2120,The Tortilla Spot,"The Tortilla Spot, Phoenix, AZ 85033",0x872b15e5e565d97b:0xf8d844c3de2d44d0,None,33.493299,-112.218176,[Restaurant],4.0,28,None,"[[Tuesday, 10AM–8PM], [Wednesday, 10AM–8PM], [...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 8PM ⋅ Opens 10AM Wed,"[0x872b154f290ece1b:0xc8081e971a081bd3, 0x872b...",https://www.google.com/maps/place//data=!4m2!3...,True


In [64]:
df_metasit.drop_duplicates(subset=['gmap_id'], inplace=True)

In [65]:
df_metasit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210953 entries, 0 to 212013
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              210953 non-null  object 
 1   address           209900 non-null  object 
 2   gmap_id           210953 non-null  object 
 3   description       72174 non-null   object 
 4   latitude          210953 non-null  float64
 5   longitude         210953 non-null  float64
 6   category          210953 non-null  object 
 7   avg_rating        210953 non-null  float64
 8   num_of_reviews    210953 non-null  int64  
 9   price             99738 non-null   object 
 10  hours             189013 non-null  object 
 11  MISC              209549 non-null  object 
 12  state             189848 non-null  object 
 13  relative_results  170277 non-null  object 
 14  url               210953 non-null  object 
 15  duplex            210953 non-null  bool   
dtypes: bool(1), float64(3), i

***Hay 1053 LOCALES SIN DIRECCIÓN*** :  se les coloca temporalmente el valor de ''

In [66]:
df_metasit.address = df_metasit.address.fillna('')

***SEPARACIÓN de la columna ADDRESS en 3 columnas: Calle-Num-Ciudad, Estado2, CP***

In [67]:
df_metasit[['NOMBRE','DIRECCION']] = df_metasit.address.str.split(', ', n=1, expand=True)
df_metasit[['CALLE-NUM-CIUDAD', 'CODIGOPOSTAL5']] = df_metasit.DIRECCION.str.rsplit(', ', n=1, expand=True)
df_metasit[['ESTADO2', 'CP']] = df_metasit.CODIGOPOSTAL5.str.split(' ', n=1, expand=True)

In [68]:
df_metasit[['name','address','CALLE-NUM-CIUDAD', 'CODIGOPOSTAL5','ESTADO2', 'CP']].sample(5)

,name,address,CALLE-NUM-CIUDAD,CODIGOPOSTAL5,ESTADO2,CP
107532,Shell,"Shell, 6400 N Henney Rd, Coyle, OK 73027","6400 N Henney Rd, Coyle",OK 73027,OK,73027
86952,517 Wingz,"517 Wingz, 312 Brown St, Uvalda, GA 30473","312 Brown St, Uvalda",GA 30473,GA,30473
52354,New York Bagel Airline,"New York Bagel Airline, 13200 Airline Hwy, Bat...","13200 Airline Hwy, Baton Rouge",LA 70817,LA,70817
33562,Garrett Fountain (Garrett),"Garrett Fountain (Garrett), 6241 KY-550, Garre...","6241 KY-550, Garrett",KY 41630,KY,41630
39394,Hawaiian Bros Island Grill,"Hawaiian Bros Island Grill, 4030 S Noland Rd, ...","4030 S Noland Rd, Independence",MO 64055,MO,64055


In [69]:
df_metasit.drop(columns=['NOMBRE','DIRECCION','CODIGOPOSTAL5','duplex'],inplace=True)
df_metasit.address = df_metasit.address.replace('',pd.NA)

In [74]:
df_metasit.name = df_metasit.name.str.title()
df_metasit.MISC = df_metasit.MISC.str.lower()
df_metasit.category = df_metasit.category.str.lower()

In [81]:
df_metasit[df_metasit.state=='Permanently closed']

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,CALLE-NUM-CIUDAD,ESTADO2,CP
2,"Sweet Rewards Gluten Free Bakery, Llc","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,NaN,4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...",NaN,Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...,"LLC, 85 NE Dartmoor Dr, Waukee",IA,50263
6,Fresh Point Country Buffet,"Fresh Point Country Buffet, 10525 US-19, Pinel...",0x88c2e4e34f1ed783:0x76c5da381c499d79,None,27.867489,-82.702971,NaN,5.0,2,None,"[[Thursday, 11AM–9PM], [Friday, 11AM–9PM], [Sa...",NaN,Permanently closed,"[0x88c2e5593fbd022b:0x2ccd7a719849c9f7, 0x88c2...",https://www.google.com/maps/place//data=!4m2!3...,"10525 US-19, Pinellas Park",FL,33782
10,The Aynor Inn,"The Aynor Inn, 343 8th Ave, Aynor, SC 29511",0x890007d78c3e5ab9:0x4f105096aaef191d,None,33.996785,-79.202560,NaN,4.3,2,None,"[[Thursday, 11AM–2PM], [Friday, 11AM–2PM], [Sa...",NaN,Permanently closed,"[0x890007d7258551b9:0xf7aae370beb052c6, 0x8900...",https://www.google.com/maps/place//data=!4m2!3...,"343 8th Ave, Aynor",SC,29511
11,Paula'S Kitchen,"Paula's Kitchen, 777 Elmwood Ave, Providence, ...",0x89e44fe1c2a2a2c3:0x5fc9f6751fb29737,None,41.790994,-71.422631,NaN,4.7,18,None,"[[Thursday, 11AM–7PM], [Friday, 11AM–7PM], [Sa...",NaN,Permanently closed,None,https://www.google.com/maps/place//data=!4m2!3...,"777 Elmwood Ave, Providence",RI,02907
19,Off The Hoof,"Off The Hoof, 201 E 4th St, Santa Ana, CA 92701",0x80dcd95d192d988b:0x68795f58e35bf888,None,33.748329,-117.866045,NaN,4.0,3,None,"[[Thursday, 11AM–10PM], [Friday, 11AM–10PM], [...",NaN,Permanently closed,None,https://www.google.com/maps/place//data=!4m2!3...,"201 E 4th St, Santa Ana",CA,92701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211998,Mac'S Bbq,"Mac's BBQ, 630 Hollywood Dr, Jackson, TN 38301",0x887ea119d9e8fd09:0x17df17c499c93f0d,BBQ sandwiches & plates with typical sides dis...,35.622428,-88.835994,NaN,4.5,58,None,"[[Monday, 10AM–8PM], [Tuesday, 10AM–8PM], [Wed...",NaN,Permanently closed,"[0x887ea0d73f82fd15:0x70928f5e82f18da0, 0x887e...",https://www.google.com/maps/place//data=!4m2!3...,"630 Hollywood Dr, Jackson",TN,38301
211999,Jackson Pizza,"Jackson Pizza, 81 Cassville Rd, Jackson Townsh...",0x89c17b3b65b4680d:0x40a43d960a97a0,None,40.109785,-74.387818,NaN,4.5,28,None,"[[Monday, 11AM–9PM], [Tuesday, 11AM–9PM], [Wed...",NaN,Permanently closed,None,https://www.google.com/maps/place//data=!4m2!3...,"81 Cassville Rd, Jackson Township",NJ,08527
212000,Royal Bavarian Schnitzel Haus,"Royal Bavarian Schnitzel Haus, 1085 NE 79th St...",0x88d9b3cd3f274101:0x64335c60e4e3065e,Old-world Bavarian food hall with a menu of tr...,25.848295,-80.174799,NaN,4.3,144,$$,"[[Monday, 5–11PM], [Tuesday, 6–11PM], [Wednesd...",NaN,Permanently closed,"[0x88d9b42010ea8cc1:0x564f1011f4820906, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...,"1085 NE 79th St, Miami",FL,33138
212001,Panheads Pizzeria,"Panheads Pizzeria, 4085 S Ridgewood Ave, Port ...",0x88e6d78a11694343:0xaed512a21b391dcd,None,29.142263,-80.986379,NaN,4.4,98,None,"[[Monday, 4–9PM], [Tuesday, 4–9PM], [Wednesday...",NaN,Permanently closed,"[0x88e6d7fe1cee97f5:0xe18598f350e28d5a, 0x88e6...",https://www.google.com/maps/place//data=!4m2!3...,"4085 S Ridgewood Ave, Port Orange",FL,32127


#### ***<u>COMENTARIOS</u>***
- Hay 49978 LOCALES CERRADOS, se deberían eliminar esos registros.
- Las columnas **state** porque no tiene sentido y **price** deberían eliminarse porque la cantidades de datos nulos sobrepasa los 50%.

*Se obtuvo las columnas de Calle-Num-Ciudad, Estado2, CP(código postal) para posteriores análisis*

#### *3. REVIEWS-ESTADOS*

In [84]:
df_reviewest = pd.read_parquet(r"Generated\Google\reviews-estados.parquet")

In [85]:
df_reviewest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8339179 entries, 0 to 8339178
Data columns (total 10 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
 8   anio     int64 
 9   estado   object
dtypes: int64(3), object(7)
memory usage: 636.2+ MB


#### ***~ VERIFICACIÓN DE DUPLICADOS***

In [86]:
df_reviewest['duplex'] = df_reviewest.duplicated(subset=['user_id','gmap_id'])
df_reviewest[df_reviewest.duplex]

,user_id,name,time,rating,text,pics,resp,gmap_id,anio,estado,duplex
59,100463062858079561066,Michelle Salangsang,1554848008177,5,Its a Fabulous place I love it!,None,None,0x809b2037e088215d:0xbd3cebbdb6533638,2019,California,True
92,105158284063533917070,M K,1559952948240,1,"Worst place, slow and they don’t know what the...",None,{'text': 'Sorry your experience wasn't better....,0x80dce8b37f4132fb:0xd392d6512a804ac7,2019,California,True
222,108988389924947764866,Anton Perreau,1567223861308,5,Love this place. It's a little set back from t...,None,{'text': 'Thankyou! We love Having you in the ...,0x80c2c6594138441d:0x59088f7293270c3f,2019,California,True
224,112636981749878576043,Aiste Vitkauske,1528408432219,5,Daniella is such a friendly and professional h...,None,{'text': 'Thank you so much! We can't wait to ...,0x80c2c6594138441d:0x59088f7293270c3f,2018,California,True
290,107274210617592949422,danny mendez,1493247531381,5,The cut hair the way i like it.,None,None,0x80c2b5b48e09708d:0x7cc44e4a125e8bad,2017,California,True
...,...,...,...,...,...,...,...,...,...,...,...
7776866,114905297462563185798,Rebecca Myhre,1563483114753,5,"Love our Gemma, just wish visiting other dogs ...",[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c414bbe9741f61:0x2256924349ae59d7,2019,Pennsylvania,True
7776947,104345676049955102357,Karl Land,1548712834930,5,We love it and our dog does too woof,None,None,0x89c4161dfac40e6d:0x4ec7760114996467,2019,Pennsylvania,True
7777082,102876110863730082589,Andrea Petaccio,1488454713514,3,Housewares can be a bit pricey and clothing is...,None,None,0x89c6c00a398b9211:0x1cfa9cce250174cb,2017,Pennsylvania,True
7777090,118308225041707512971,Dave Knight,1505085573561,3,"average thrift shop, tough to look at some ite...",None,None,0x89c6c00a398b9211:0x1cfa9cce250174cb,2017,Pennsylvania,True


#### ***<u>COMENTARIO IMPORTANTE</u>***
- Este dataset se es MUY SIMILAR al 1° MERGE_SITE_REVIEWS, por eso me parece que se debe ELIMINAR.

### **YELP**

#### *1. BUSINESS_CHECKIN*

In [3]:
df_businesschk = pd.read_parquet(r"Generated\Yelp\business_checkin.parquet")

In [4]:
df_businesschk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   3330 non-null   object 
 1   name          3330 non-null   object 
 2   address       3330 non-null   object 
 3   city          3330 non-null   object 
 4   state         3330 non-null   object 
 5   postal_code   3330 non-null   object 
 6   latitude      3330 non-null   float64
 7   longitude     3330 non-null   float64
 8   stars         3330 non-null   float64
 9   review_count  3330 non-null   int64  
 10  is_open       3330 non-null   int64  
 11  attributes    3296 non-null   object 
 12  categories    3330 non-null   object 
 13  hours         3082 non-null   object 
 14  date          3330 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 390.4+ KB


In [5]:
df_businesschk.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,date
1822,VDFajBnUgEkeYU2P2HXK-w,Brew n Burger,28530 US Hwy 19 N,Clearwater,FL,33761,28.031578,-82.738751,3.0,20,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Hot Dogs, Sandwiches, Restaurants, American (T...","{'Friday': '11:0-23:0', 'Monday': '11:0-23:0',...","2019-04-05 01:17:11, 2019-04-05 23:40:05, 2019..."
2313,_RPipScNmjl_aTVyDIx94A,Reme's Restaurant,"2683 E Main St, Ste 113",Plainfield,FL,46168,39.716305,-86.353746,4.5,47,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Pizza, Pancakes, American (Traditional), Resta...","{'Friday': '7:0-21:0', 'Monday': None, 'Saturd...","2019-07-06 17:28:08, 2019-07-14 16:14:21, 2019..."
2477,S-aqWmXiRT2XYCLYugdBdQ,Zeko's Mediterranean Grill,806 E Hillsborough Ave,Tampa,PA,33604,27.996356,-82.452434,4.0,58,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Greek, Seafood, Chicken Wings, Ve...","{'Friday': '10:0-4:0', 'Monday': '10:0-4:0', '...","2017-02-02 17:07:15, 2017-03-10 01:58:38, 2017..."


In [6]:
df_businesschk.categories = df_businesschk.categories.str.lower()

#### ***~ VERIFICACIÓN DE DUPLICADOS***

In [12]:
df_businesschk['duplex'] = df_businesschk.duplicated(subset=['business_id'])
df_businesschk[df_businesschk.duplex]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,date,duplex


#### ***<u>COMENTARIOS</u>***
- Este dataset está casi depurado, sólo se cambió el campo **categories**.
- No hubo duplicados.
- La columna **is_open** debería eliminarse.

#### *2. BUSINESS_TIP*

In [9]:
df_businesstip = pd.read_parquet(r"Generated\Yelp\business_tip.parquet")

In [10]:
df_businesstip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69346 entries, 0 to 69345
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           69346 non-null  object 
 1   business_id       69346 non-null  object 
 2   text              69346 non-null  object 
 3   date              69346 non-null  object 
 4   compliment_count  69346 non-null  int64  
 5   name              69346 non-null  object 
 6   address           69346 non-null  object 
 7   city              69346 non-null  object 
 8   state             69346 non-null  object 
 9   postal_code       69346 non-null  object 
 10  latitude          69346 non-null  float64
 11  longitude         69346 non-null  float64
 12  stars             69346 non-null  float64
 13  review_count      69346 non-null  int64  
 14  is_open           69346 non-null  int64  
 15  attributes        69301 non-null  object 
 16  categories        69346 non-null  object

In [11]:
df_businesstip.sample(3)

,user_id,business_id,text,date,compliment_count,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
56091,cttKJ5bUCugUrtQZ4uy-Ww,WeIEAM8QEDHF_EM-8mzeIA,So simple \nFair price \nDelicious \nNice retr...,2019-07-12 00:00:41,0,Simple Ice Cream Sandwiches,960 S Virginia St,Reno,CA,89502,39.515431,-119.807489,4.5,138,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Ice Cream & Frozen Yogurt, Restaurants, Sandwi...","{'Friday': '11:30-21:0', 'Monday': '11:30-20:0..."
35602,r9gEc0zaB1KWv551uxGG3w,12GUv17IX7dJbuzeBxBCyg,Great food and drinks\nService is very good\nT...,2019-03-25 04:42:42,0,J J's Pie,555 W 5th St,Reno,PA,89503,39.530070,-119.822050,4.0,252,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Italian, Event Planning & Services, Pizza, Cat...","{'Friday': '11:0-23:0', 'Monday': '11:0-22:0',..."
26951,MwQdiBm7DE4DFAkpeSadKw,-1b2kNOowsPrPpBOK4lNkQ,Love it,2017-09-16 12:31:31,0,Usaquén,1700 N 3rd St,Philadelphia,PA,19122,39.976618,-75.140844,4.5,22,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Breakfast & Brunch, Colombian, Restaurants, La...","{'Friday': '18:0-22:0', 'Monday': None, 'Satur..."


In [15]:
df_businesstip.text[56091]

'So simple \nFair price \nDelicious \nNice retro decor'

In [ ]:
df_businesstip.text = df_businesstip.text.str.lower()
df_businesstip.categories = df_businesstip.categories.str.lower()

#### ***~ VERIFICACIÓN DE DUPLICADOS***

In [13]:
df_businesstip['duplex'] = df_businesstip.duplicated(subset=['user_id','business_id'])
df_businesstip[df_businesstip.duplex]

,user_id,business_id,text,date,compliment_count,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,duplex
484,fCvMnJU1Z-XhAjKg99wK3Q,NQbGOi0OUWt_-ra_nles0g,Now open.,2017-03-20 11:23:01,0,Davio's Galleria,150 Main St,King of Prussia,CA,19406,40.083863,-75.404571,3.0,25,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Gelato, Food, Coffee & Tea, Break...","{'Friday': '7:0-21:0', 'Monday': '7:0-19:0', '...",True
593,sLgekUp1uedWpnDsztvkVw,j_fmYfswAHKZqucuUS1J9Q,It's happy hour ! I guess I'm on a margaritas ...,2017-10-15 21:52:54,0,El Nopal,3 Stonegate Ctr,Valley Park,PA,63088,38.566338,-90.495285,3.5,87,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Mexican, Restaurants","{'Friday': '11:0-22:0', 'Monday': '11:0-20:30'...",True
620,fCvMnJU1Z-XhAjKg99wK3Q,ndVqbK8tlQSZM63rajPkBQ,"""What happens at Palizzi stays at Palizzi.\nNo...",2017-02-07 14:23:14,0,Palizzi Filippo Social Club,1408 S 12th St,Philadelphia,PA,19147,39.932157,-75.164411,4.5,15,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Social Clubs, Arts & Entertainment, Restaurant...",None,True
627,WzhoLpHMXI59ETuQ2sRVeQ,t3PkHzVTvPA8qRxKe9TIkQ,Come on Tuesday they have free live music. Mon...,2017-08-20 15:31:28,0,Booker's Restaurant and Bar,5021 Baltimore Ave,Philadelphia,FL,19143,39.948041,-75.224239,3.5,276,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Bars, Event Planning & Services, Nightlife, Sa...","{'Friday': '10:0-22:0', 'Monday': '0:0-0:0', '...",True
637,ftiitD4ua2PbMsKfs43LRg,mYMPepp0QIZRk_52pWzeoA,"Just go, relax and enjoy. Particularly fond of...",2018-03-25 04:38:31,0,Seasons 52,204 N Westshore Blvd,Tampa,PA,33609,27.946370,-82.524812,4.0,701,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","American (Traditional), Gluten-Free, Steakhous...","{'Friday': '11:0-23:0', 'Monday': '0:0-0:0', '...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69174,0syX0E_7RoLAiplX5MKNxQ,wbwDE3g8j4w-TGy_E6dMSw,Great tasting food. Meat was perfect and the s...,2019-02-25 20:04:42,0,Cazador Grill,10918 N 56th St,Temple Terrace,FL,33617,28.048109,-82.394241,4.5,259,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Steakhouses, Peruvian, Seafood","{'Friday': '16:0-22:0', 'Monday': '16:0-21:0',...",True
69195,W-6fBv6nnFgAR8kzP2mOyg,oBNrLz4EDhiscSlbOl8uAw,Eggs Bennie deliciousness,2017-12-29 19:29:29,0,Ruby Slipper - New Orleans,200 Magazine St,New Orleans,FL,70130,29.951025,-90.067394,4.5,5193,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, American (Traditional), American ...","{'Friday': '7:30-14:0', 'Monday': '0:0-0:0', '...",True
69223,Yr-fuc95s18LpQXN1u71OA,qISf5ojuYbD9h71NumGUQA,Amazing! Food was fast and fantastic!,2019-10-07 07:57:02,0,Han Dynasty,123 Chestnut St,Philadelphia,FL,19106,39.948526,-75.143728,4.0,1474,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Beer, Wine & Spirits, Chinese, Restaurants, Fo...","{'Friday': '12:0-22:0', 'Monday': '16:0-21:30'...",True
69253,0_EoyM242PqCYtFUoD0eTg,1b5mnK8bMnnju_cvU65GqQ,BONUTS!!! So delicious! No crowd on Thursday l...,2019-08-03 04:51:40,0,Biscuit Love: Gulch,316 11th Ave S,Nashville,PA,37203,36.153050,-86.783231,4.0,4207,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Burgers, American (Traditional), Breakfast & B...","{'Friday': '7:0-15:0', 'Monday': '0:0-0:0', 'S...",True


In [16]:
df_businesstip.drop(columns=['duplex'])
df_businesstip.drop_duplicates(subset=['user_id','business_id'], inplace=True)

#### ***<u>COMENTARIOS</u>***
- Este dataset está casi depurado, sólo se cambiaron los campos **text** y **categories**.
- Se borraron 5006 duplicados.
- La columna **is_open** debería eliminarse.

#### *3. BUSINESS*

In [18]:
df_business = pd.read_parquet(r"Generated\Yelp\bussines.parquet")

In [19]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22685 entries, 3 to 150339
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   22685 non-null  object 
 1   name          22685 non-null  object 
 2   address       22685 non-null  object 
 3   city          22685 non-null  object 
 4   state         22685 non-null  object 
 5   postal_code   22685 non-null  object 
 6   latitude      22685 non-null  float64
 7   longitude     22685 non-null  float64
 8   stars         22685 non-null  float64
 9   review_count  22685 non-null  int64  
 10  is_open       22685 non-null  int64  
 11  attributes    22429 non-null  object 
 12  categories    22685 non-null  object 
 13  hours         19504 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 2.6+ MB


In [20]:
df_business.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3378,Y4ekZBFgSauDaVOqI52vPQ,Blue Coast Burrito Express,2323 Elliston Pl,Nashville,PA,37203,36.148946,-86.806162,3.5,8,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Mexican","{'Friday': '11:0-21:0', 'Monday': '11:0-21:0',..."
116178,SpQTj-YIQDq2_3IiMq5BXw,Tavola Dei Amice,1603 Kings Hwy N,Cherry Hill,PA,08034,39.915668,-75.008410,3.5,6,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Pizza, Restaurants, Italian",None
95109,hPxDTFxMGBIrXAiJeWgFcw,KFC,895 N Green St,Brownsburg,FL,46112,39.855235,-86.392272,1.5,8,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Chicken Wings, Chicken Shop, Fast...","{'Friday': '10:0-22:0', 'Monday': '10:0-22:0',..."


In [21]:
df_business.categories = df_business.categories.str.lower()

#### ***~ VERIFICACIÓN DE DUPLICADOS***

In [22]:
df_business['duplex'] = df_business.duplicated(subset=['business_id'])
df_business[df_business.duplex]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,duplex


#### ***<u>COMENTARIOS</u>***
- Este dataset está casi depurado, sólo se cambió el campos **categories**.
- No hubo duplicados.
- La columna **is_open** debería eliminarse.